In [7]:
from HTML import config as Config
from HTML.dataset import *
from HTML.config import ratio, nominal, ordinal, meaningless
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from HTML import preprocessing as ps
%matplotlib inline
data = pd.read_csv(Config.train_path)
data = ps.preprocessing(data)
data.head()


,ID,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,0,Resort Hotel,0,342,2015,July,27,1,0,0,...,No Deposit,None,None,0,Transient,-6.305161,0,0,Check-Out,2015-07-01
1,1,City Hotel,0,257,2015,July,27,1,0,2,...,No Deposit,6,None,0,Transient,75.052227,0,0,Check-Out,2015-07-03
2,2,City Hotel,0,257,2015,July,27,1,0,2,...,No Deposit,6,None,0,Transient,74.546401,0,0,Check-Out,2015-07-03
3,3,City Hotel,0,257,2015,July,27,1,0,2,...,No Deposit,6,None,0,Transient,76.376288,0,0,Check-Out,2015-07-03
4,4,City Hotel,0,257,2015,July,27,1,0,2,...,No Deposit,6,None,0,Transient,49.411647,0,0,Check-Out,2015-07-03


In [8]:
data = data.drop(['reservation_status','reservation_status_date', 'adr'], axis=1)
data = pd.concat([data, 
                 pd.get_dummies(data['hotel'], drop_first=True), 
                 pd.get_dummies(data['arrival_date_month'], drop_first=True), 
                 pd.get_dummies(data['market_segment'], drop_first=True),
                 pd.get_dummies(data['distribution_channel'], drop_first=True),
                 pd.get_dummies(data['customer_type'], drop_first=True), 
                 pd.get_dummies(data['deposit_type'], drop_first=True),
                 pd.get_dummies(data['meal'], drop_first=True),
                 pd.get_dummies(data['country'], drop_first=True),
                 pd.get_dummies(data['reserved_room_type'], drop_first=True),
                 pd.get_dummies(data['assigned_room_type'], drop_first = True)
                 ], axis=1)
data = data.drop(['assigned_room_type', 'reserved_room_type', 'meal', 'country', 'hotel','arrival_date_month','market_segment','distribution_channel','customer_type', 'deposit_type'], axis=1)
data = data.drop(['ID', 'agent', 'company'], axis=1)
data.head()

,is_canceled,lead_time,arrival_date_year,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,...,C,D,E,F,G,H,I,K,L,P
0,0,342,2015,27,1,0,0,2,0.0,0,...,1,0,0,0,0,0,0,0,0,0
1,0,257,2015,27,1,0,2,1,0.0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,257,2015,27,1,0,2,2,0.0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,257,2015,27,1,0,2,2,0.0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,257,2015,27,1,0,2,2,0.0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91527 entries, 0 to 91526
Columns: 231 entries, is_canceled to P
dtypes: float64(1), int64(16), uint8(214)
memory usage: 30.6 MB


In [10]:
from sklearn.model_selection import cross_val_score
def CreateBalancedSampleWeights(y_train, largest_class_weight_coef):
    classes = np.unique(y_train)
    classes.sort()
    class_samples = np.bincount(y_train)
    total_samples = class_samples.sum()
    n_classes = len(class_samples)
    weights = total_samples / (n_classes * class_samples * 1.0)
    class_weight_dict = {key : value for (key, value) in zip(classes, weights)}
    class_weight_dict[classes[1]] = class_weight_dict[classes[1]] * largest_class_weight_coef
    sample_weights = [class_weight_dict[y] for y in y_train]

    return sample_weights



In [ ]:
# Fit and train
s = []
x = data[set(data.columns) - {'is_canceled'}]
y = data['is_canceled']
from sklearn.svm import SVC
clf = SVC(kernel = 'rbf', random_state=0)
scores = cross_val_score(clf, x, y, cv=5, scoring='f1', verbose=1, n_jobs=2)
s.append(scores)
print(np.mean(scores))
# Predict
# y_pred = classifier.predict(X_test)

# Computing accuracy
# model_accuracy_results['RandomForest'] = model_accuracy(y_test, y_pred)

[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.


In [ ]:
plt.plot([1,5,10,50,100,500], np.mean(np.array(s),axis=1))